# Install library

In [ ]:
%pip install -q pip==23.3.2
%pip install -q gym_super_mario_bros nes_py
%pip install -q wheel==0.38.4 setuptools==66.0.0 gym==0.21.0
%pip install -q numpy==1.26.4 opencv-python==4.8.0.76

In [ ]:
%pip install -q torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%pip install -q stable-baselines3[extra]==1.5.0

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Setup environment and wrappers


In [ ]:
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from nes_py.wrappers import JoypadSpace
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.monitor import Monitor

In [ ]:
from gym import RewardWrapper

class CustomRewardWrapper(RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.curr_score = 0

    def reset(self, **kwargs):
        self.curr_score = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        reward += (info["score"] - self.curr_score) / 40.

        self.curr_score = info["score"]

        if done:
            if info["flag_get"]:
                reward += 50
            else:
                reward -= 50

        return state, reward/10, done, info

In [ ]:
from gym import Wrapper

class SkipFrame(Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            state, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return state, total_reward, done, info

In [ ]:
def create_env():
    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = CustomRewardWrapper(env)
    env = SkipFrame(env, skip=4)
    env = GrayScaleObservation(env, keep_dim=True)
    env = ResizeObservation(env,    (84, 84))
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, 4, channels_order='last')
    return env

In [ ]:
env = create_env()

# Training

In [ ]:
import os 
from stable_baselines3 import PPO

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
model = PPO(
    'CnnPolicy',
    env,
    verbose=1,
    learning_rate=0.0001,
    n_steps=512,
    device ="auto",
    gae_lambda=1.0,
    ent_coef=0.01,
    gamma=0.9,
    batch_size=64,
    n_epochs=10,
    tensorboard_log=LOG_DIR
)

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
# import numpy as np

class TrainingProgressMonitor(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.model.num_timesteps % self.check_freq == 0 and self.model.num_timesteps > 0:
            model_path = os.path.join(self.save_path, 'model_{}_steps'.format(self.model.num_timesteps))
            self.model.save(model_path)  
        return True

In [ ]:
callback = TrainingProgressMonitor(check_freq=200000, save_path=CHECKPOINT_DIR)

In [ ]:
model.learn(total_timesteps=5_000_000, callback=callback)

# Test

In [ ]:
model = PPO.load('./train/model_5000000_steps', env=env)

In [ ]:
import time

state = env.reset()
num_epsisodes = 10
total_rewards = 0
total_steps = 0
total_flags = 0
i = 0
try:
    while i < num_epsisodes:
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        total_rewards += reward
        total_steps += 1
        if info[0].get("flag_get"):
            total_flags += 1
        env.render()
        time.sleep(1/100)
        if done:
            state = env.reset()
            i += 1
    print("Number of episodes:", num_epsisodes)
    print(f"Mean reward: {total_rewards/num_epsisodes}")
    print(f"Mean steps: {total_steps/num_epsisodes}")
    print(f"Total flags: {total_flags}")
    env.close()

except KeyboardInterrupt:
    print("\nInterrupted by user. Closing environment...")
    env.close()
    env = create_env()